In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from mxnet import nd, autograd, gluon
import mxnet as mx
from mxnet.gluon import nn, rnn

In [2]:
def n(digits =3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)

def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

N = 50000
N_train = int(N * .9)
N_validation = N - N_train

digits = 3
input_digits = digits * 2 + 3
output_digits = digits + 3

added = set()
questions = []
answers = []
answers_y = []

while len(questions) < N:
    a, b = n(), n()
    pair = tuple(sorted((a, b)))
    if pair in added:
        continue
        
    question = 'S{}+{}E'.format(a, b)
    question = padding(question, input_digits)
    answer = 'S' + str(a + b) + 'E'
    answer = padding(answer, output_digits)
    answer_y = str(a + b) + 'E'
    answer_y = padding(answer_y, output_digits)
    
    added.add(pair)
    questions.append(question)
    answers.append(answer)
    answers_y.append(answer_y)
    
chars = '0123456789+SE '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 3, len(chars)), dtype=np.integer)
Z = np.zeros((len(questions), digits + 3, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers_y[i]):
        Z[i, t, char_indices[char]] = 1
    
X_train, X_validation, Y_train, Y_validation, Z_train, Z_validation = \
    train_test_split(X, Y, Z, train_size=N_train)

/opt/venv/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Testset

In [3]:
def gen_n_test(N):
    q = []
    y = []
    for i in range(N):
        a, b = n(), n() 
        question = '{}+{}'.format(a, b)
        answer_y = str(a + b)
        q.append(question)
        y.append(answer_y)
    return(q,y)

In [4]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [17]:
class calculator(gluon.Block):
    def __init__(self, n_hidden, in_seq_len, out_seq_len, vocab_size, enc_layer, dec_layer = 1, **kwargs):
        super(calculator, self).__init__(**kwargs)
        self.in_seq_len = in_seq_len
        self.out_seq_len = out_seq_len
        self.n_hidden = n_hidden
        self.vocab_size = vocab_size
        self.enc_layer = enc_layer
        
        with self.name_scope():
            self.encoder = rnn.LSTM(hidden_size = n_hidden, num_layers = enc_layer, layout = 'NTC')
            self.decoder_0 = rnn.LSTMCell(hidden_size = n_hidden)
            self.decoder_1 = rnn.LSTMCell(hidden_size = n_hidden)
            self.batchnorm = nn.BatchNorm(axis = 2)
            self.dense = nn.Dense(self.vocab_size, flatten = False)
            
    def forward(self, inputs, outputs):
        # API says: num_layers, batch_size, num_hidden
        self.batch_size = inputs.shape[0]
        begin_state = self.encoder.begin_state(batch_size = self.batch_size, ctx = ctx)
        enout, (h, c) = self.encoder(inputs, begin_state) # h, c: n_layer * batch_size * n_hidden
        # Pick the hidden states and cell states at the last time step in the second layer
        next_h_0 = h[0] # batch_size * n_hidden
        next_c_0 = c[0] # batch_size * n_hidden
        next_h_1 = h[1] # batch_size * n_hidden
        next_c_1 = c[1] # batch_size * n_hidden
        for i in range(self.out_seq_len):
            deout, (next_h_0, next_c_0) = self.decoder_0(outputs[:, i, :], [next_h_0, next_c_0],)
            deout, (next_h_1, next_c_1) = self.decoder_1(deout, [next_h_1, next_c_1],)
            if i == 0:
                deouts = deout
            else:
                deouts = nd.concat(deouts, deout, dim = 1)   
        deouts = nd.reshape(deouts, (-1, self.out_seq_len, self.n_hidden))
        deouts = self.batchnorm(deouts)
        deouts_fc = self.dense(deouts)
        return deouts_fc
    
    def calculation(self, input_str, char_indices, indices_char, input_digits = 9, lchars = 14, ctx = mx.gpu()):
        input_str = 'S' + input_str + 'E'
        X = nd.zeros((1, input_digits, lchars), ctx = ctx)
        for t, char in enumerate(input_str):
            X[0, t, char_indices[char]] = 1
        Y_init = nd.zeros((1, lchars), ctx = ctx)
        Y_init[0, char_indices['S']] = 1
        begin_state = self.encoder.begin_state(batch_size = 1, ctx = ctx)
        enout, (h, c) = self.encoder(X, begin_state)
        next_h_0 = h[0] # batch_size * n_hidden
        next_c_0 = c[0] # batch_size * n_hidden
        next_h_1 = h[1] # batch_size * n_hidden
        next_c_1 = c[1] # batch_size * n_hidden
        deout = Y_init
        
        for i in range(self.out_seq_len):
            deout, (next_h_0, next_c_0) = self.decoder_0(deout, [next_h_0, next_c_0],)
            deout, (next_h_1, next_c_1) = self.decoder_1(deout, [next_h_1, next_c_1],)
            deout = nd.expand_dims(deout, axis = 1)
            deout = self.batchnorm(deout)
            deout = deout[:, 0, :]
            deout_sm = self.dense(deout)
            deout = nd.one_hot(nd.argmax(nd.softmax(deout_sm, axis = 1), axis = 1), depth = self.vocab_size)
            if i == 0:
                ret_seq = indices_char[nd.argmax(deout_sm, axis = 1).asnumpy()[0].astype('int')]
            else:
                ret_seq += indices_char[nd.argmax(deout_sm, axis = 1).asnumpy()[0].astype('int')]

            if ret_seq[-1] == ' ' or ret_seq[-1] == 'E':
                break
        return ret_seq.strip('E').strip()
        

In [18]:
tr_set = gluon.data.ArrayDataset(X_train, Y_train, Z_train)
tr_data_iterator = gluon.data.DataLoader(tr_set, batch_size=256, shuffle=True)

te_set =gluon.data.ArrayDataset(X_validation, Y_validation, Z_validation)
te_data_iterator = gluon.data.DataLoader(te_set, batch_size=256, shuffle=True)

In [19]:
ctx = mx.gpu()
model = calculator(300, 9, 6, 14, 2)
model.collect_params().initialize(mx.init.Xavier(), ctx = ctx)

trainer = gluon.Trainer(model.collect_params(), 'rmsprop')
loss = gluon.loss.SoftmaxCrossEntropyLoss(axis = 2, sparse_label = False)

In [20]:
print(model)

calculator(
  (encoder): LSTM(None -> 300, NTC, num_layers=2)
  (decoder_0): LSTMCell(None -> 1200)
  (decoder_1): LSTMCell(None -> 1200)
  (batchnorm): BatchNorm(axis=2, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
  (dense): Dense(None -> 14, linear)
)


In [21]:
def calculate_loss(model, data_iter, loss_obj, ctx = ctx):
    test_loss = []
    for i, (x_data, y_data, z_data) in enumerate(data_iter):
        x_data = x_data.as_in_context(ctx).astype('float32')
        y_data = y_data.as_in_context(ctx).astype('float32')
        z_data = z_data.as_in_context(ctx).astype('float32')
        with autograd.predict_mode():
            z_output = model(x_data, y_data)
            loss_te = loss_obj(z_output, z_data)
        curr_loss = nd.mean(loss_te).asscalar()
        test_loss.append(curr_loss)
    return np.mean(test_loss)

In [22]:
epochs = 201

tot_test_loss = []
tot_train_loss = []
for e in range(epochs):
    train_loss = []
    for i, (x_data, y_data, z_data) in enumerate(tr_data_iterator):
        x_data = x_data.as_in_context(ctx).astype('float32')
        y_data = y_data.as_in_context(ctx).astype('float32')
        z_data = z_data.as_in_context(ctx).astype('float32')
        
        with autograd.record():
            z_output = model(x_data, y_data)
            loss_ = loss(z_output, z_data)
        loss_.backward()
        trainer.step(x_data.shape[0])
        curr_loss = nd.mean(loss_).asscalar()
        train_loss.append(curr_loss)
        
    if e % 10 == 0:
        q, y = gen_n_test(10)
        for i in range(10):
            with autograd.predict_mode():
                p = model.calculation(q[i], char_indices, indices_char).strip()
                iscorr = 1 if p == y[i] else 0
                if iscorr == 1:
                    print(colors.ok + '☑' + colors.close, end=' ')
                else:
                    print(colors.fail + '☒' + colors.close, end=' ')
                print("{} = {}({}) 1/0 {}".format(q[i], p, y[i], str(iscorr) ))
    #caculate test loss
    test_loss = calculate_loss(model, te_data_iterator, loss_obj = loss, ctx=ctx) 

    print("Epoch %s. Train Loss: %s, Test Loss : %s" % (e, np.mean(train_loss), test_loss))    
    tot_test_loss.append(test_loss)
    tot_train_loss.append(np.mean(train_loss))


☒ 4+83 = 103(87) 1/0 0
☒ 32+6 = 132(38) 1/0 0
☒ 23+35 = 338(58) 1/0 0
☒ 804+140 = 1003(944) 1/0 0
☒ 501+37 = 550(538) 1/0 0
☒ 48+6 = 153(54) 1/0 0
☒ 1+2 = 122(3) 1/0 0
☒ 307+595 = 1003(902) 1/0 0
☒ 36+3 = 133(39) 1/0 0
☒ 4+4 = 553(8) 1/0 0
Epoch 0. Train Loss: 1.1887326, Test Loss : 1.137922
Epoch 1. Train Loss: 1.1229072, Test Loss : 1.1939143
Epoch 2. Train Loss: 1.0980244, Test Loss : 1.1030667
Epoch 3. Train Loss: 1.0271647, Test Loss : 1.0102959
Epoch 4. Train Loss: 0.91535574, Test Loss : 0.85927093
Epoch 5. Train Loss: 0.836573, Test Loss : 0.8067001
Epoch 6. Train Loss: 0.76906264, Test Loss : 0.72716874
Epoch 7. Train Loss: 0.68128014, Test Loss : 0.6523584
Epoch 8. Train Loss: 0.5845852, Test Loss : 0.5454272
Epoch 9. Train Loss: 0.50449175, Test Loss : 0.52240574
☒ 580+46 = 625(626) 1/0 0
☒ 6+6 = 133(12) 1/0 0
☑ 5+5 = 10(10) 1/0 1
☒ 116+50 = 167(166) 1/0 0
☒ 86+9 = 174(95) 1/0 0
☑ 554+51 = 605(605) 1/0 1
☒ 154+0 = 155(154) 1/0 0
☑ 6+660 = 666(666) 1/0 1
☒ 464+649 = 1013(1113

Epoch 87. Train Loss: 7.3070696e-06, Test Loss : 0.0011937881
Epoch 88. Train Loss: 7.1502695e-06, Test Loss : 0.0011608541
Epoch 89. Train Loss: 6.8019885e-06, Test Loss : 0.0011796046
☑ 7+41 = 48(48) 1/0 1
☑ 8+5 = 13(13) 1/0 1
☑ 9+53 = 62(62) 1/0 1
☒ 86+9 = 175(95) 1/0 0
☑ 5+12 = 17(17) 1/0 1
☒ 8+2 = 11(10) 1/0 0
☒ 6+7 = 134(13) 1/0 0
☑ 711+717 = 1428(1428) 1/0 1
☑ 8+450 = 458(458) 1/0 1
☑ 63+44 = 107(107) 1/0 1
Epoch 90. Train Loss: 6.6332636e-06, Test Loss : 0.001151774
Epoch 91. Train Loss: 6.4999517e-06, Test Loss : 0.0012712192
Epoch 92. Train Loss: 6.4856195e-06, Test Loss : 0.0011744685
Epoch 93. Train Loss: 6.3154544e-06, Test Loss : 0.0011834592
Epoch 94. Train Loss: 6.149737e-06, Test Loss : 0.0011855698
Epoch 95. Train Loss: 6.130489e-06, Test Loss : 0.0012341099
Epoch 96. Train Loss: 5.953523e-06, Test Loss : 0.0011711314
Epoch 97. Train Loss: 5.920632e-06, Test Loss : 0.0011936727
Epoch 98. Train Loss: 5.6574936e-06, Test Loss : 0.001177714
Epoch 99. Train Loss: 5.703460

Epoch 171. Train Loss: 2.3104656e-06, Test Loss : 0.0010896982
Epoch 172. Train Loss: 2.3356392e-06, Test Loss : 0.0010612385
Epoch 173. Train Loss: 2.2933402e-06, Test Loss : 0.0010352186
Epoch 174. Train Loss: 2.2688014e-06, Test Loss : 0.0010203301
Epoch 175. Train Loss: 2.2571119e-06, Test Loss : 0.001018396
Epoch 176. Train Loss: 2.1932153e-06, Test Loss : 0.0010605085
Epoch 177. Train Loss: 2.1797202e-06, Test Loss : 0.0010207925
Epoch 178. Train Loss: 2.1409842e-06, Test Loss : 0.001036236
Epoch 179. Train Loss: 2.1663159e-06, Test Loss : 0.0010475998
☑ 398+68 = 466(466) 1/0 1
☑ 4+669 = 673(673) 1/0 1
☑ 8+847 = 855(855) 1/0 1
☑ 52+26 = 78(78) 1/0 1
☑ 425+7 = 432(432) 1/0 1
☒ 992+7 = 1000(999) 1/0 0
☑ 8+931 = 939(939) 1/0 1
☑ 44+678 = 722(722) 1/0 1
☑ 4+59 = 63(63) 1/0 1
☑ 3+4 = 7(7) 1/0 1
Epoch 180. Train Loss: 2.1899762e-06, Test Loss : 0.0010465644
Epoch 181. Train Loss: 2.1507333e-06, Test Loss : 0.0010328765
Epoch 182. Train Loss: 2.1278631e-06, Test Loss : 0.0010787027
Epoc

MXNetError: [00:15:38] src/storage/./pooled_storage_manager.h:119: cudaMalloc failed: out of memory

Stack trace returned 10 entries:
[bt] (0) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x34aee2) [0x7f077e35aee2]
[bt] (1) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x34b4a8) [0x7f077e35b4a8]
[bt] (2) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x3533a4f) [0x7f0781543a4f]
[bt] (3) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x353853c) [0x7f078154853c]
[bt] (4) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e59ca0) [0x7f0780e69ca0]
[bt] (5) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e59f17) [0x7f0780e69f17]
[bt] (6) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2e5a319) [0x7f0780e6a319]
[bt] (7) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2dc09f2) [0x7f0780dd09f2]
[bt] (8) /opt/venv/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x2dc09f2) [0x7f0780dd09f2]
[bt] (9) /opt/venv/lib